In [15]:
import os
import sys
print os.environ['JAVA_HOME']
#os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars /data/spark-csv_2.11-1.5.0.jar'
spark_home = os.environ['SPARK_HOME'] = '/root/spark15'
if not spark_home:
    raise ValueError('SPARK_HOME enviroment variable is not set')
sys.path.insert(0,os.path.join(spark_home,'python'))
sys.path.insert(0,os.path.join(spark_home,'python/lib/py4j-0.9-src.zip'))
execfile(os.path.join(spark_home,'python/pyspark/shell.py'))
sc.setLogLevel("WARN")


/opt/jdk1.8.0_144
Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.2.0
      /_/

Using Python version 2.7.3 (default, Feb 12 2013 14:01:46)
SparkSession available as 'spark'.


In [14]:

del os.environ['PYSPARK_SUBMIT_ARGS']

In [16]:
fileout = sc.textFile("/user/root/w210/full_text/patent_claims_fulltext.csv")

In [17]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)
df = sqlContext.read.csv( path = "/user/root/w210/full_text/patent_claims_fulltext.csv",header = True,inferSchema = True)


### SPARK : Getting Data Schema

In [20]:
df.printSchema()

root
 |-- pat_no: integer (nullable = true)
 |-- claim_no: integer (nullable = true)
 |-- claim_txt: string (nullable = true)
 |-- dependencies: string (nullable = true)
 |-- ind_flg: integer (nullable = true)
 |-- appl_id: integer (nullable = true)



### Let's see Few sample claim_txt


In [44]:
df.select("claim_txt").head(10)

[Row(claim_txt=u'1. A golf glove comprising at least an index finger receptacle, a middle finger receptacle and a back surface extending from said receptacles in the direction of the wrist, and further comprising another finger receptacle disposed adjacent said index finger receptacle and said middle finger receptacle and overlapping a portion of said back section.'),
 Row(claim_txt=u"4. A golf glove adapted for use on one hand of a golf player comprising at least an index finger receptacle and further comprising a finger receptacle attached to the side of said index finger receptacle most nearly adjacent to the thumb and adapted to receive a finger of the golf player's other hand."),
 Row(claim_txt=u'3. A glove comprising an index finger receptacle, a middle finger receptacle, a back surface extending in the direction of the wrist, a finger restraining strap extending between said receptacles, and a finger restraining strap attached to said back surface adjacent said index finger rece

### Get total number of records

In [21]:
df.count()

52192948

### Get the Summary statistics

In [23]:
df.describe()

DataFrame[summary: string, pat_no: string, claim_no: string, claim_txt: string, dependencies: string, ind_flg: string, appl_id: string]

### Find Unique Patent Cases

In [24]:
df.select('pat_no').distinct().count()

3537528

### Find records with no NULL

In [25]:
df.dropna().count()

41028838

###Average Length of claim Text

In [28]:
from pyspark.sql.functions import length

dfnew = df.withColumn('length', length("claim_txt"))

In [37]:
dfnew.groupby('pat_no').agg({'length': 'sum'}).head(4)


[Row(pat_no=3931035, sum(length)=710),
 Row(pat_no=3931157, sum(length)=3134),
 Row(pat_no=3931434, sum(length)=840),
 Row(pat_no=3931856, sum(length)=1615)]

In [38]:
ndf = dfnew.groupby('pat_no').agg({'length': 'sum'})

In [40]:
ndf.columns

['pat_no', 'sum(length)']

#### Maximum Length of claim text

In [41]:
from pyspark.sql.functions import max
ndf.select(max('sum(length)')).show()

+----------------+
|max(sum(length))|
+----------------+
|          479210|
+----------------+



#### Minimum Length

In [42]:
from pyspark.sql.functions import min
ndf.select(min('sum(length)')).show()

+----------------+
|min(sum(length))|
+----------------+
|              11|
+----------------+



In [43]:
from pyspark.sql.functions import mean
ndf.select(mean('sum(length)')).show()

+-----------------+
| avg(sum(length))|
+-----------------+
|5660.391970042358|
+-----------------+

